# Le rapprochement de chaînes de caractères

Avez-vous déjà essayer de rapprocher des chaînes de caractères de manière automatique ?

Très rapidement vous vous retrouverez face à un certain nombre de problèmes difficiles à gérer.

Voici un exemple de 4 chaînes très proches :

In [1]:
chaine1 = "Python chez AXA"
chaine2 = "python chez axa"
chaine3 = "a...Python chez AXA...a"
chaine4 = "Pyton chez AXA"

Essayez de transformer ces chaînes de façon à pouvoir écrire sans erreur :

In [2]:
# assert permet de tester une expression et renvoie une erreur en cas 
# de non égalité
# assert chaine1 == chaine2 == chaine3 == chaine4

## La distance de Levenshtein

La distance de Levenshtein est une métrique permettant de mesurer l'écart entre deux séquences de mots. 

En d'autres termes, elle mesure le nombre minimum de modifications que vous devez effectuer pour modifier une séquence d'un mot en un autre. Ces modifications peuvent être des insertions, des suppressions ou des substitutions. Cette mesure a été nommée en l'honneur de Vladimir Levenshtein, qui l'avait envisagée en 1965.

La définition en python de la distance de Levenshtein entre deux chaînes a et b peut être vue comme suit:

In [3]:
import numpy as np
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "Les chaînes sont à {} changements près".format(distance[row][col])

Essayez de calculer la distance entre les chaînes précédentes dans une matrice :

In [4]:
print(eval("chaine"+"1"))

Python chez AXA


In [5]:
levenshtein_ratio_and_distance(chaine1,chaine3,ratio_calc=True)

0.7894736842105263

Il s'agit ici d'une implémentation manuelle de la distance de Levenshtein. Il existe un package Levenshtein pour la calculer automatiqmement :

Utilisez le fichier `.whl` adapté pour votre version de Python en utilisant :

`pip install .../....whl`

In [6]:
import Levenshtein as lev

In [7]:
Distance = lev.distance(chaine1.lower(),chaine4.lower()),
print(Distance)
Ratio = lev.ratio(chaine1.lower(),chaine4.lower())
print(Ratio)

(1,)
0.9655172413793104


## Aller plus loin : le package de référence FuzzyWuzzy

`pip install fuzzywuzzy`

ou alors avec le fichier `.whl`



In [8]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

On peut calculer d'autres distances :

In [9]:
print(fuzz.ratio(chaine1, chaine3))
# 97
print(fuzz.partial_ratio(chaine1, chaine3))
# 92

79
100


On peut utiliser des tokens :

In [10]:
print(fuzz.token_sort_ratio("python chez Axa", chaine3))
# 92
fuzz.token_set_ratio('Barack Obama', 'Barack H. Oama')
# 100

88


88

Si on a une liste, on peut utiliser :

In [11]:
query = 'Barack Obama'
choices = ['Barack H Obama', 'Barack H. Obama', 'B. Obama']
# Get a list of matches ordered by score, default limit to 5
process.extract(query, choices)
# [('Barack H Obama', 95), ('Barack H. Obama', 95), ('B. Obama', 85)]
 
# If we want only the top one
process.extractOne(query, choices)
# ('Barack H Obama', 95)

('Barack H Obama', 95)

In [13]:
query = "SARL/U"
choices = ["SA","SASU", "EURL", "SAS" ]

In [14]:
process.extract(query, choices, scorer= fuzz.token_sort_ratio)

[('SASU', 60), ('SA', 50), ('SAS', 44), ('EURL', 40)]

In [15]:
process.extract(query, choices, scorer= fuzz.ratio,limit=2)

[('SASU', 60), ('SA', 50)]

In [16]:
process.extractOne(query, choices, scorer= fuzz.ratio, score_cutoff=70)

Nous verrons l'application sur des données de ces outils dans la partie sur pandas.

In [17]:
import string 
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'